In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [12]:
att=pd.read_csv('/Users/david/final/attributes.csv')
desc=pd.read_csv('/Users/david/final/product_descriptions.csv')
train=pd.read_csv('/Users/david/final/train.csv')
test=pd.read_csv('/Users/david/final/test.csv')

In [13]:
att = att.fillna(0)
att['product_uid'] = att['product_uid'].astype(np.int64)
att['string']=att['name'].map(str)+' '+att['value'].map(str)
att

,product_uid,name,value,string
0,100001,Bullet01,Versatile connector for various 90° connection...,Bullet01 Versatile connector for various 90° c...
1,100001,Bullet02,Stronger than angled nailing or screw fastenin...,Bullet02 Stronger than angled nailing or screw...
2,100001,Bullet03,Help ensure joints are consistently straight a...,Bullet03 Help ensure joints are consistently s...
3,100001,Bullet04,Dimensions: 3 in. x 3 in. x 1-1/2 in.,Bullet04 Dimensions: 3 in. x 3 in. x 1-1/2 in.
4,100001,Bullet05,Made from 12-Gauge steel,Bullet05 Made from 12-Gauge steel
...,...,...,...,...
2044798,224428,Pilot Bit Included,No,Pilot Bit Included No
2044799,224428,Power Tool Product Type,Drill & Router Bits,Power Tool Product Type Drill & Router Bits
2044800,224428,Shank Diameter,3/8 in,Shank Diameter 3/8 in
2044801,224428,Shank Style,Hex,Shank Style Hex


In [14]:
att1=pd.pivot_table(att,index=['product_uid'],values=['string'],aggfunc=lambda x: ' '.join(x))
att1

,string
product_uid,
0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
100001,Bullet01 Versatile connector for various 90° c...
100002,"Application Method Brush,Roller,Spray Assemble..."
100003,Built-in flange Yes Bullet01 Slightly narrower...
100004,Amperage (amps) 8.56 Bullet01 Positive power t...
...,...
224422,Assembled Depth (in.) 0.44 in Assembled Height...
224423,Bullet01 Adjusts to fit shower openings with a...
224424,Assembly Required No Bullet01 Solid wood const...


In [15]:
train=pd.merge(train, desc, how='left', on='product_uid')
train=pd.merge(train, att1, how='left', on='product_uid')
train

,id,product_uid,product_title,search_term,relevance,product_description,string
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they ...",Bullet01 Versatile connector for various 90° c...
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they ...",Bullet01 Versatile connector for various 90° c...
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,BEHR Premium Textured DECKOVER is an innovativ...,"Application Method Brush,Roller,Spray Assemble..."
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Update your bathroom with the Delta Vero Singl...,Bath Faucet Type Combo Tub and Shower Built-in...
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Update your bathroom with the Delta Vero Singl...,Bath Faucet Type Combo Tub and Shower Built-in...
...,...,...,...,...,...,...,...
74062,221457,206638,Atlantic Windowpane 576 CD or 192 DVD Blu-Ray ...,tv riser glass,1.00,"Atlantic, Inc. 94835722 Uniquely designed for ...",Assembled Depth (in.) 9.5 in Assembled Height ...
74063,221458,206639,Philips 40-Watt Halogen R20 Flood Light Bulb (...,r20 halogen light,3.00,Philips Energy Advantage lamps use less energy...,Actual Color Temperature (K) 2700 Average Life...
74064,221463,206641,Schlage Camelot In-Active Aged Bronze Handlese...,schlage lock siena half dummy knob with,2.33,The Schlage Camelot In-Active Aged Bronze Hand...,Application Type Entry Assembled Depth (in.) 2...
74065,221471,206648,Plastec 11 in. x 24 in. Rose Garden Wall Decor...,zen garden decor,3.00,The Rose Garden is inspired by the popular ear...,Bullet01 Indoor and outdoor use Bullet02 Finis...


In [16]:
train['string'] = train['string'].replace(np.nan, '', regex=True)
train['term_desc'] = [len(set(a.split()) & set(b.split())) for a, b in zip(train.search_term, train.product_description)]
train['term_att'] = [len(set(a.split()) & set(b.split())) for a, b in zip(train.search_term, train.string)]
train['term_prod'] = [len(set(a.split()) & set(b.split())) for a, b in zip(train.search_term, train.product_title)]
#calculate ratios here
train['sum']=train['term_desc']+train['term_att']
train['q_length']=[len(a.split()) for a in train['search_term']]
train['ratio1']=train['term_prod']/train['q_length']
train['ratio2']=train['sum']/train['q_length']
train.sort_values('relevance')

,id,product_uid,product_title,search_term,relevance,product_description,string,term_desc,term_att,term_prod,sum,q_length,ratio1,ratio2
12889,39841,109485,Home Legend Wire Brushed Natural Hickory 3/8 i...,lock brushed nicke;,1.0,Wire Brushed 3/8 in. High Density Fiberboard (...,Board Form Plank Bullet01 Wood Species: Hickor...,0,0,0,0,3,0.00,0.000000
55533,168244,166858,Winworks Wood Composite 12 in. x 40 in. Raised...,40 yr roofing shingle,1.0,Winworks premium shutters are custom crafted t...,Bullet01 Wood composite construction up to 1-1...,0,1,1,1,4,0.25,0.250000
41545,126511,143687,Home Accents Holiday 5 ft. Pre-Lit Tinsel Sant...,home for lease signs,1.0,Greet your guests this Christmas season with t...,"Bullet01 Weather-resistant, soft tinsel materi...",1,1,0,2,4,0.00,0.500000
69729,209076,196207,Carlisle 6.56 in. Diameter Melamine Narrow Rim...,6 rim,1.0,"The Durus line offers high quality, heavyweigh...",Bullet01 Exclusive mirror-like glaze gives sup...,0,0,0,0,2,0.00,0.000000
55531,168239,166855,SimTek Ashland 6 in. x 9 in. Red Cedar Composi...,Propane line plug,1.0,Ashland by SimTek Fence has reinvented fence m...,Bullet01 Made with polyethylene plastic and de...,1,1,0,2,3,0.00,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37599,114998,138143,Galcon 7101D Waterproof Battery Operated Contr...,battery drip valve,3.0,The Galcon 7101D is a waterproof battery opera...,,2,0,0,2,3,0.00,0.666667
37597,114996,138141,Crown Bolt 1/4 in.-20 x 4 in. Stainless Hanger...,hanger bolt,3.0,These hanger bolts are headless bolts designed...,,1,0,0,1,2,0.00,0.500000
37595,114983,138134,Jeffrey Court Tiffany May 3 in. x 6 in. Glass ...,Jeffrey Court Tile,3.0,"With its smooth glossy surface and high sheen,...",Approximate Tile Size 3x6 Break Strength (lb.)...,3,3,3,6,3,1.00,2.000000
37630,115068,138181,Nostalgia Electrics Coca-Cola Series Slush Mac...,snow cone,3.0,The Coca-Cola Slush Drink Maker from Nostalgia...,,0,0,0,0,2,0.00,0.000000


In [17]:
test=pd.merge(test, desc, how='left', on='product_uid')
test=pd.merge(test, att1, how='left', on='product_uid')

test['string'] = test['string'].replace(np.nan, '', regex=True)
test['term_desc'] = [len(set(a.split()) & set(b.split())) for a, b in zip(test.search_term, test.product_description)]
test['term_att'] = [len(set(a.split()) & set(b.split())) for a, b in zip(test.search_term, test.string)]
test['term_prod'] = [len(set(a.split()) & set(b.split())) for a, b in zip(test.search_term, test.product_title)]
test['sum']=test['term_desc']+test['term_att']
test['q_length']=[len(a.split()) for a in test['search_term']]
test['ratio1']=test['term_prod']/test['q_length']
test['ratio2']=test['sum']/test['q_length']

In [18]:
ready=train[['relevance','term_desc','term_att','sum','term_prod','q_length','ratio1','ratio2']]
test1=test[['id','term_desc','term_att','sum','term_prod','q_length','ratio1','ratio2']]

In [19]:
X = ready.loc[:, ready.columns != 'relevance']
y = ready.loc[:, ready.columns == 'relevance']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

rg= Ridge(alpha=.1)
rg.fit(X_train, y_train.values.ravel())
y_pred = rg.predict(X_test)
rg_mse = mean_squared_error(y_pred, y_test)
rg_rmse = np.sqrt(rg_mse)
print('Ridge RMSE: %.4f' % rg_rmse)

Ridge RMSE: 0.5191


In [20]:
Xt = test1.loc[:, test1.columns != 'id']
y_pred = rg.predict(Xt)
y_pred

array([2.39570057, 2.28497106, 2.28497106, ..., 2.38912034, 2.46619682,
       2.34587671])

In [21]:
results=pd.DataFrame()
results['id']=test1['id']
results['relevance']=y_pred
results.to_csv("submission.csv", index=False)